# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [567]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [568]:
driver = webdriver.Chrome()
driver.get("https://jportal.mdcourts.gov/license/pbIndex.jsp?ac=y")


**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [569]:
submit_button = driver.find_element_by_id('checkbox')
submit_button.send_keys('yes')
submit_button.click()
botton = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
botton.click()





Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [570]:
driver.get("https://jportal.mdcourts.gov/license/pbPublicSearch.jsp")

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [571]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_name('slcJurisdiction'))
select.select_by_visible_text('Statewide')

In [572]:
text_input = driver.find_element_by_id('txtTradeName')
text_input.send_keys(' VAP%')


And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [573]:
submit_button = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]")
submit_button.submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [574]:
name = driver.find_elements_by_class_name('tablecelltext')[0].find_elements_by_tag_name('td')[1]
name.text

'AMIN NARGIS'

In [575]:
name = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[4]/td[2]').text
name

'AMIN NARGIS'

In [576]:
name2 = driver.find_elements_by_class_name('tablecelltext')[4].find_elements_by_tag_name('td')[1].text
name2

name2 = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[9]/td[2]').text
name2

'AMIN NARGIS'

In [577]:
names = driver.find_elements_by_class_name('searchfieldtitle')

for name in names:
    each = name.find_element_by_class_name('searchlistitem').text
    print(each)



VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [578]:
names = driver.find_elements_by_class_name('searchfieldtitle')

for name in names:
    stores = name.find_element_by_class_name('searchlistitem').text
 
    try:
        detail_button = name.find_element_by_tag_name('a')
        print(stores)
        print(detail_button.get_attribute('href')) 
    
    except:
        print("The shop's status is Pending")     



VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D
The shop's status is Pending
The shop's status is Pending


Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

In [ ]:
# names = driver.find_elements_by_class_name('searchfieldtitle')

# for name in names:
#     stores = name.find_element_by_class_name('searchlistitem').text
 
#     try:
#         detail_button = name.find_element_by_tag_name('a')
# #         print(stores) 
# #         print(detail_button.get_attribute('href')) 

#         url = detail_button.get_attribute('href')
#         driver.get(url)
        
#         mail_address = driver.find_element_by_class_name('tablecelltext').find_element_by_tag_name('td')
#         location_address = driver.find
        
        
        
#         return_button = driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[4]/tbody/tr/td[1]/a')
#         return_button.click()
        

        
        
        
        
        
#     except:
#         print(stores)   


### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [579]:
names = driver.find_elements_by_class_name('searchfieldtitle')

shops = []
for name in names:
    shop = {}  
    
    shop['stores'] = name.find_element_by_class_name('searchlistitem').text
    
    try:
        detail_button = name.find_element_by_tag_name('a')
        shop['url'] = detail_button.get_attribute('href')
        
    except:
        
        shop
        
    shops.append(shop)

print(shops)

[{'stores': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D'}, {'stores': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D'}, {'stores': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D'}, {'stores': 'VAPE FROG'}, {'stores': 'VAPE FROG'}]


In [580]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(shops)
df.head()

,stores,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN


### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [581]:
names = driver.find_elements_by_class_name('searchfieldtitle')

for name in names:
    stores = name.find_element_by_class_name('searchlistitem').text
 
    try:
        detail_button = name.find_element_by_tag_name('a')
        print(stores) 
        print(detail_button.get_attribute('href')) 
        
                    
    except:
        print(stores)  

next_button = driver.find_element_by_partial_link_text('Next')
driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
next_button.click()

VAPE IT STORE II
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
VAPE IT STORE I
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
VAPEPAD THE
https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D
VAPE FROG
VAPE FROG


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [ ]:
# while True:
#     names = driver.find_elements_by_class_name('searchfieldtitle')

#     shops = []
#     for name in names:
#         shop = {}  

#         shop['stores'] = name.find_element_by_class_name('searchlistitem').text

#         try:
#             detail_button = name.find_element_by_tag_name('a')
#             shop['url'] = detail_button.get_attribute('href')

#         except:

#             shop

#         shops.append(shop)

#     print(shops)   
    
#     next_button = driver.find_element_by_partial_link_text('Next')
#     driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
#     next_button.click()
    
#     print("Scraping a new page")

### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [588]:
shops = []
try:

    while True:
        names = driver.find_elements_by_class_name('searchfieldtitle')

        for name in names:
            shop = {}  

            shop['stores'] = name.find_element_by_class_name('searchlistitem').text

            try:
                detail_button = name.find_element_by_tag_name('a')
                shop['url'] = detail_button.get_attribute('href')

            except:

                shop

            shops.append(shop)

        print(shops)   

        next_button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", next_button)
        next_button.click()

        print("Scraping a new page")
except:
    pass
   
    

[{'stores': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D'}, {'stores': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D'}, {'stores': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D'}, {'stores': 'VAPE FROG'}, {'stores': 'VAPE FROG'}]
Scraping a new page
[{'stores': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D'}, {'stores': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D'}, {'stores': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D'}, {'stores': 'VAPE FROG'}, {'stores': 'VAPE FROG'}, {'stores': 'VAPE LOFT THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ySR%2Fg2G%2BQ1s%3D'}, {'stores': 'VAPE N CIGAR', 'url': 'https://jportal.mdcourts.gov

Scraping a new page
[{'stores': 'VAPE IT STORE II', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D'}, {'stores': 'VAPE IT STORE I', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D'}, {'stores': 'VAPEPAD THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D'}, {'stores': 'VAPE FROG'}, {'stores': 'VAPE FROG'}, {'stores': 'VAPE LOFT THE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ySR%2Fg2G%2BQ1s%3D'}, {'stores': 'VAPE N CIGAR', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2nfkPkWxPO8%3D'}, {'stores': 'VAPE DOJO', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Kbs6EoQNlVg%3D'}, {'stores': 'VAPE HAVEN'}, {'stores': 'VAPORS LOUNGE'}, {'stores': 'VAPORFI SOUTHSIDE', 'url': 'https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wuOSUN2R8fA%3D'}, {'stores': 'VAPE DO JO', 'url': 'https://jportal.mdcourts.gov/l

In [589]:
len(shops)

34

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [590]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(shops)
df

,stores,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ySR%2Fg2G%2BQ1s%3D
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2nfkPkWxPO8%3D
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Kbs6EoQNlVg%3D
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


In [591]:
df.to_csv("vape-shops-basic.csv", index=False)

In [592]:
pd.read_csv("vape-shops-basic.csv")

,stores,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN
5,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ySR%2Fg2G%2BQ1s%3D
6,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2nfkPkWxPO8%3D
7,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Kbs6EoQNlVg%3D
8,VAPE HAVEN,NaN
9,VAPORS LOUNGE,NaN


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [593]:
df.url.head(2)

0    https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
1    https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
Name: url, dtype: object

### Use Selenium to visit that page

In [595]:
driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=g7qsFXl5BN8%3D")
select = Select(driver.find_element_by_name('slcJurisdiction'))
select.select_by_visible_text('Statewide')

text_input = driver.find_element_by_id('txtTradeName')
text_input.send_keys(' VAP%')

submit_button = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]")
submit_button.submit()
driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D")


### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

In [596]:
mailing_add = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]").text
location_add = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td").text
license_info = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]").text.split('\n')
price = driver.find_elements_by_class_name('rt')[-1].text
issue = driver.find_elements_by_class_name("tablecelltext")[-2].text


In [597]:
license_detail = {}

license_detail['mailing_add'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]").text
license_detail['location_add'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td").text
license_detail['license_info'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]").text.split('\n')
license_detail['price'] = driver.find_elements_by_class_name('rt')[-1].text
license_detail['issue'] = driver.find_elements_by_class_name("tablecelltext")[-2].text

license_detail 


{'mailing_add': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location_add': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'license_info': ['License Status: Issued',
  'License No.: 22375606',
  'Control No.: 22884439',
  'Date of Issue: 4/27/2018',
  'Months Paid: 12',
  'Exp. Date: 4/30/2019',
  'Subdivision: 22 Salisbury'],
 'price': '$ 41.00',
 'issue': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

In [599]:
driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=g7qsFXl5BN8%3D")
select = Select(driver.find_element_by_name('slcJurisdiction'))
select.select_by_visible_text('Statewide')

text_input = driver.find_element_by_id('txtTradeName')
text_input.send_keys(' VAP%')

submit_button = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]")
submit_button.submit()
driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D")

license_detail = {}

license_detail['mailing_add'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]").text
license_detail['location_add'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td").text
license_detail['license_info'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]").text.split('\n')
license_detail['price'] = driver.find_elements_by_class_name('rt')[-1].text
license_detail['issue'] = driver.find_elements_by_class_name("tablecelltext")[-2].text

license_detail 

{'mailing_add': 'AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804',
 'location_add': 'VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801',
 'license_info': ['License Status: Issued',
  'License No.: 22375606',
  'Control No.: 22884439',
  'Date of Issue: 4/27/2018',
  'Months Paid: 12',
  'Exp. Date: 4/30/2019',
  'Subdivision: 22 Salisbury'],
 'price': '$ 41.00',
 'issue': 'WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198'}

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [602]:
def get_detail(row):
    try:
        driver.get("https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=g7qsFXl5BN8%3D")
        select = Select(driver.find_element_by_name('slcJurisdiction'))
        select.select_by_visible_text('Statewide')

        text_input = driver.find_element_by_id('txtTradeName')
        text_input.send_keys(' VAP%')

        submit_button = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/form/table/tbody/tr[14]/td/input[1]")
        submit_button.submit()
        driver.get(row['url'])

        license_detail = {}

        license_detail['mailing_add'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]").text
        license_detail['location_add'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td").text
        license_detail['license_info'] = driver.find_element_by_xpath("/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]").text.split("\n")
        
        license_detail['price'] = driver.find_elements_by_class_name('rt')[-1].text
        license_detail['issue'] = driver.find_elements_by_class_name("tablecelltext")[-2].text


        return pd.Series(license_detail)
    except:
        return pd.Series({})

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [603]:
new_df = df.apply(get_detail, axis=1).join(df)
new_df

,issue,license_info,location_add,mailing_add,price,stores,url
0,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[License Status: Issued, License No.: 22375606, Control No.: 22884439, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE II\n1015 S SALISBURY BLVD\nSALISBURY, MD 21801","AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804",$ 41.00,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=wHYJYrZ4Jl0%3D
1,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198","[License Status: Issued, License No.: 22375605, Control No.: 22591855, Date of Issue: 4/27/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 22 Salisbury]","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801","AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804",$ 24.00,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=z5icXfFdDJM%3D
2,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401","[License Status: Issued, License No.: 02304705, Control No.: 02685930, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","VAPEPAD THE\n2299 JOHNS HOPKINS ROAD\nGAMBRILLS, MD 21054","ANJ DISTRIBUTIONS LLC\n2504 ORCHARD KNOLL WAY\nODENTON, MD 21113",$ 94.00,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=1x9FWcM4FtA%3D
3,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
4,NaN,NaN,NaN,NaN,NaN,VAPE FROG,NaN
5,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401","[License Status: Issued, License No.: 02310799, Control No.: 02686069, Date of Issue: 4/03/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 02 Anne Arundel County]","VAPE LOFT THE\n185 MITCHELLS CHANCE RD\nEDGEWATER, MD 21037",DISBROW II EMERSON HARRINGTON,$ 154.00,VAPE LOFT THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=ySR%2Fg2G%2BQ1s%3D
6,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045","[License Status: Issued, License No.: 13343011, Control No.: 13856368, Date of Issue: 4/30/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 13 Howard County]","VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045",DISCOUNT TOBACCO ESSEX LLC,$ 84.00,VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=2nfkPkWxPO8%3D
7,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155","[License Status: Issued, License No.: 06327188, Control No.: 06946760, Date of Issue: 4/05/2018, Months Paid: 12, Exp. Date: 4/30/2019, Subdivision: 06 Westminster]","VAPE DOJO\n330 ONE FORTY VILLAGE ROAD\nUNIT 15\nWESTMINSTER, MD 21157",FAIRGROUND VILLAGE LLC,$ 179.00,VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=Kbs6EoQNlVg%3D
8,NaN,NaN,NaN,NaN,NaN,VAPE HAVEN,NaN
9,NaN,NaN,NaN,NaN,NaN,VAPORS LOUNGE,NaN


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [ ]:
new_df.to_csv("vape-total.csv", index=False)

In [ ]:
pd.read_csv("vape-total.csv")